# Variational Quantum Circuits

***

## Intro

Quantum algorithms based on the idea of **variational quantum circuits** can be executed on near-term quantum devices to tackle some optimization problems$^1$.

What is variational quantum circuit? After reading this post, you will get an answer to the question thanks to intuitive explanation and a simple example in Python [PennyLane](https://pennylane.readthedocs.io/en/latest/index.html) library from Xanadu.

***

## Variational Circuit

Variational circuits are a family of **hybrid quantum-classical** algorithms$^2$. The ingredients of a variational circuit are as follows:

- **Parameterized quantum circuit** $U(x,\theta)$,
- **Output** from the quantum circuit,
- **Learning algorithm** and objective function with respect to which, the parameterized unitary is trained.

<img src="https://i.imgur.com/yu3rGb0.png" title="The ingredients of variational circuit." style="width: 600px;"/>

We will go through each of these elements separately.


### Parameterized quantum circuit

Parameterized quantum circuit $U(x,\theta)$ is a unitary transformation that is $\theta$-parameterized ($\theta$ is a vector) and $x$-parameterized ($x$ is an input vector).

<img src="https://i.imgur.com/Yz4CrOg.png" title="Variational quantum circuit is parameterized by input x and free parameters theta." style="width: 250px;"/>

We can decompose the unitary $U(x,\theta)$ into a set of smaller unitaries. An **example** of such variational quantum circuit with $\theta=[\theta_1, \theta_2, \theta_3]^T$ is presented below:

<img src="https://i.imgur.com/wviLmen.png" title="An example of variational quantum circuit. Note that there are many ways of how you construct variational circuit architecture." style="width: 500px;"/>

where $U_0(x)$ encodes an input data $x$ into a quantum state. Unitaries such as $U_2(\theta_2)$ are parameterized quantum gates, for instance $U_2(\theta_2)=R_X(\theta_2)$, which is a single-qubit rotation through angle $\theta_2$ around the x-axis (on Bloch sphere).

<img src="https://i.imgur.com/0qzyniG.gif" title="Rotation around x axis for theta ∈ [0,2π]." style="width: 300px;"/>

Take note that variational circuits can also contain gates which are not parameterized (like $H$, $\textit{CNOT}$, etc.).


### Output

How do we extract information from $U(x,\theta)$, i.e. how quantum information is converted into classical one?

Firstly, we measure an observable $\hat{B}$ at the output. However, a single measurement would not give much information in this case. Therefore, we repeat the measurement many times (each time we prepare the quantum circuit from scratch) to get an **estimate of expectation value**, which is a function of $x$ and $\theta$:

\begin{align}
f(x,\theta) &= \langle 0 | U^{\dagger}(x,\theta)\hat{B}U(x,\theta)) |0 \rangle
\end{align}

<img src="https://i.imgur.com/gbSxUXY.png" title="Repetitive measurements constitute for an estimate of expectation value of some chosen observable." style="width: 700px;"/>

As an observable we can use a subset of [Pauli operators](https://en.wikipedia.org/wiki/Pauli_matrices) $\{\hat{B}\} \subset \{I,\sigma_x,\sigma_y,\sigma_z\}^{\otimes N}$ (for example $\sigma_z$ measured on each qubit).


### Learning algorithm

The **key insight** is that $\theta$ parameters can be adjusted so that an objective function is minimized and variational circuit produces the desired output. This sound like a supervised machine learning algorithm, and in fact variational quantum circuits can be used for that kind of purposes (see this blog post about [Quantum Circuit Learning](http://dkopczyk.quantee.co.uk/qcl/) algorithm).

Typically, variational circuits are trained using classical optimization algorithm such as [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) - this is why variational circuits belong to the hybrid quantum-classical family. The objective function is $f(x,\theta)$ usually subject to some classical post-processing.

<img src="https://i.imgur.com/vOdTw3M.png" title="Learning of variational quantum circuit involves iterative execution of quantum and classical parts of the algorithm." style="width: 600px;"/>

***

## Example

Ok, if it is still not clear what variational circuit does from theory, let's analyze a very simple example of a single-qubit variational circuit.

### Theory

We would take $U(x,\theta)$ to be just single gate circuit and single parameterized - for example qubit rotation around y-axis i.e. $R_Y(\theta)$ gate. Here, $\theta$ is just a number. Moreover, we quietly assumed that our input $x$ is directly encoded as the state $\rangle 0 |$, so that the term $x$ will disappear from our analysis.

<img src="https://i.imgur.com/EFrhTAw.png" title="Single parameterized gate circuit." style="width: 300px;"/>

The qubit after $R_Y(\theta)$ application is in the state:

\begin{align}
| \psi(\theta)\rangle&=R_Y(\theta)  |0\rangle \nonumber \\ &= \begin{bmatrix} \cos{\frac{\theta}{2}} & -\sin{\frac{\theta}{2}} \\ \sin{\frac{\theta}{2}} & \cos{\frac{\theta}{2}} \end{bmatrix}\begin{bmatrix}1 \\ 0\end{bmatrix} \nonumber \\ &= \begin{bmatrix} \cos{\frac{\theta}{2}} \\ \sin{\frac{\theta}{2}}\end{bmatrix}  \,.
\end{align}

As an output we measure the expectation value of Pauli-Z operator $\sigma_z$. If we would like to calculate the expectation value $f(\theta)$ on a piece of paper it would follows as:

\begin{align}
f(\theta) &= \langle 0 | R_Y^{\dagger}(\theta)\sigma_z R_Y(\theta) |0\rangle \nonumber \\ &= \langle \psi(\theta) |\sigma_z |\psi(\theta)\rangle \nonumber \\ &= \begin{bmatrix} \cos{\frac{\theta}{2}} & \sin{\frac{\theta}{2}}\end{bmatrix} \begin{bmatrix}1 & 0\\ 0 & -1\end{bmatrix} \begin{bmatrix} \cos{\frac{\theta}{2}} \\ \sin{\frac{\theta}{2}}\end{bmatrix} \nonumber \\ &= \cos^2\left(\frac{\theta}{2}\right) - \sin^2\left(\frac{\theta}{2}\right)
\end{align}

### Implementation

Let's implement the parameterized circuit on the classical simulator and check whether the results match theory. We will use [PennyLane](https://pennylane.readthedocs.io/en/latest/index.html#) from Xanadu, as a Python library for the simulation of hybrid quantum-classical computations. As the first step, use ```$ pip install pennylane```  or follow the instructions on how to install PennyLane library [here](https://pennylane.readthedocs.io/en/latest/installing.html).

We initialize the simulator of a quantum device with ```qml.device``` statement and define the circuit as a function that accepts params  argument and return an expectation value of observable $\sigma_z$. Take note that the function decorator ``` @qml.qnode(dev)``` is used to connect circuit definition with the device.

In [1]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device('default.qubit', wires=1)

@qml.qnode(dev)
def circuit(params):
    qml.RY(params[0], wires=0)
    return qml.expval.PauliZ(0)

print('The expectation value {}'.format(circuit([0])))
print('The expectation value {}'.format(circuit([np.pi/3])))
print('The expectation value {}'.format(circuit([np.pi])))

The expectation value 1.0
The expectation value 0.5
The expectation value -1.0


Well, the results are matching the theory, i.e. when we substitute $\theta=0$, $\theta=\pi/3$ and $\theta=\pi$ into the equation we will get the same results as in PennyLane.

Now, it is time to see how the circuit can be optimized to get the desired result. Our goal is to find such rotation that initial state $|0\rangle$ is transformed into $|1\rangle$.

<img src="https://i.imgur.com/m9etI4q.png" title="Variational circuit will be trained in order to covert |0> state into |1>." style="width: 700px;"/>

That corresponds to measuring Pauli-Z expectation value of $-1$. This is due to the fact that $|1\rangle$ is eigenvector of $\sigma_z$ with eigenvalue $\lambda=-1$. Thus, the objective function minimized would be simply $f(\theta)$.

In [2]:
def objective(var):
    return circuit(var)

We will initialize variational circuit with random $\theta$:

In [3]:
np.random.seed(2019)
initial_theta = 2*np.pi*np.random.random_sample()
init_params = np.array([initial_theta])
print('Initial objective function value {:.7f} for theta={:.2f}'.format(objective(init_params), initial_theta))

Initial objective function value 0.8216827 for theta=5.68


and use fantastic built-in PennyLane optimizer to find the minimum of the objective function as well as run the whole quantum-classical algorithm:

In [4]:
# Initilize Gradient Descent Optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 30
# set the initial parameter values
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(objective, params)
    print('Cost after step {:5d}: {: .7f}'.format(i+1, objective(params)))

print('Optimized rotation angle: {}'.format(params))

Cost after step     1:  0.6716096
Cost after step     2:  0.4259549
Cost after step     3:  0.0780398
Cost after step     4: -0.3151938
Cost after step     5: -0.6444255
Cost after step     6: -0.8447718
Cost after step     7: -0.9391644
Cost after step     8: -0.9773546
Cost after step     9: -0.9917454
Cost after step    10: -0.9970148
Cost after step    11: -0.9989236
Cost after step    12: -0.9996123
Cost after step    13: -0.9998604
Cost after step    14: -0.9999497
Cost after step    15: -0.9999819
Cost after step    16: -0.9999935
Cost after step    17: -0.9999977
Cost after step    18: -0.9999992
Cost after step    19: -0.9999997
Cost after step    20: -0.9999999
Cost after step    21: -1.0000000
Cost after step    22: -1.0000000
Cost after step    23: -1.0000000
Cost after step    24: -1.0000000
Cost after step    25: -1.0000000
Cost after step    26: -1.0000000
Cost after step    27: -1.0000000
Cost after step    28: -1.0000000
Cost after step    29: -1.0000000
Cost after ste

The optimized rotation angle $\theta \approx \pi$, which matches theoretical result (check out yourself!).

<img src="https://i.imgur.com/2qpGuC1.gif" title="Gradient descend visualized on Bloch sphere." style="width: 300px;"/>

We found a solution to our problem: $R_Y(\pi)$ would convert state $|0\rangle$ to $|1\rangle$. Of course, it just a dummy example but presents the main idea behind variational circuits.

You might ask how the gradient of the variational circuit is calculated (as we use gradient descent method), but this is actually a topic for the next post! Interested readers - check out PennyLane [documentation](https://pennylane.readthedocs.io/en/latest/concepts/autograd_quantum.html).

***

## Summary
To sum up, variational quantum circuit uses both classical machine and quantum device (actually quantum device simulation on classical machine, but we apply the same logic on real quantum device). The goal was to minimize some function and we accomplished that by iteratively asking the quantum device to evaluate a function $f(\theta)$ providing the $\theta$ parameters. These parameters were suggested by optimization algorithm executed on the classical machine.

To assess the **benefits and limitations**, we should examine the particular algorithm build on the idea of variational quantum circuit:

- [Variational-Quantum-Eigensolver](http://dkopczyk.quantee.co.uk/vqe/)
- [Quantum Circuit Learning](http://dkopczyk.quantee.co.uk/qcl/)
- [Quantum Approximate Optimization Algorithm (QAOA)](https://arxiv.org/pdf/1411.4028.pdf)
- [Quantum Neural Networks (Continuous-variable QNN)](https://arxiv.org/abs/1806.06871)
- [Quantum Generative Adversarial Networks (QGAN)](https://arxiv.org/abs/1804.08641)

***

## References

1. Nikolaj Moll et al, *Quantum optimization using variational algorithms on near-term quantum devices*, Quantum Sci. Technol. 3 (2018)
2. Jarrod R McClean, Jonathan Romero, Ryan Babbush, and Alán Aspuru-Guzik, [*The theory of variational hybrid quantum-classical algorithms*](https://arxiv.org/abs/1509.04279), New Journal of Physics, 18(2):023023, 2016.